In [1]:
import pandas as pd

In [2]:
# extracting tables from database
import sqlite3
def list_tables(database_file):
    connection = sqlite3.connect('data.db')
    cursor = connection.cursor()
    cursor.execute("""select name from sqlite_master where type = 'table';
                    """)
                    
    tables = cursor.fetchall()
    cursor.close()
    connection.close()
    table_names = [table[0] for table in tables]
    return table_names

In [3]:
# saving tables as individual dataframes
connection = sqlite3.connect('data.db')
query = """select *
        from client as cl;
        """
client = pd.read_sql_query(query, connection)
connection.close

connection = sqlite3.connect('data.db')
query = """select *
        from client_products as cl;
        """
client_products = pd.read_sql_query(query, connection)
connection.close

connection = sqlite3.connect('data.db')
query = """select *
        from balances as cl;
        """
balances = pd.read_sql_query(query, connection)
connection.close

connection = sqlite3.connect('data.db')
query = """select *
        from inv_campaign_eval as cl;
        """
inv_campaign_eval = pd.read_sql_query(query, connection)
connection.close

<function Connection.close()>

In [19]:
# creating one large table with client data (excluding balances table)
connection = sqlite3.connect('data.db')
query = """
        select cl.client_id, age, job, marital, education, gender, has_deposits, loan, has_insurance, has_mortgage
        from client as cl
        left join client_products as cp on cl.client_id = cp.client_id
        --left join balances as ba on cl.client_id = ba.client_id
        --left join inv_campaign_eval as camp on cl.client_id = camp.client_id;
        """
clients_merged = pd.read_sql_query(query, connection)
connection.close


<function Connection.close()>

In [5]:
# checking for datatypes
print(clients_merged.dtypes)

# checking for missing values
clients_merged.isnull().sum()

client_id          int64
age              float64
job               object
marital           object
education         object
gender            object
client_id          int64
has_deposits      object
loan              object
has_insurance     object
has_mortgage      object
client_id        float64
poutcome          object
dtype: object


client_id           0
age               232
job               438
marital             0
education           0
gender              0
client_id           0
has_deposits        0
loan                0
has_insurance       0
has_mortgage        0
client_id        8863
poutcome         8863
dtype: int64

In [13]:
balances['date'] = pd.to_datetime(balances['date'])
balances['date'] = balances['date'].dt.date
balances

,date,balance,client_id,currency
0,2023-06-06,2680.467773,249789938,CZK
1,2023-05-23,2654.055419,249789938,CZK
2,2023-05-09,2620.835872,249789938,CZK
3,2023-04-25,2720.315925,249789938,CZK
4,2023-04-11,2687.601982,249789938,CZK
...,...,...,...,...
290207,2022-08-16,145.454161,705801685,CZK
290208,2022-08-02,-5.497494,705801685,CZK
290209,2022-07-19,-45.891607,705801685,CZK
290210,2022-07-05,19.473946,705801685,CZK


In [20]:
clients_merged

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage
0,249789938,38.0,services,married,secondary,M,yes,no,n,no
1,1222646323,46.0,services,divorced,unknown,M,no,no,n,no
2,451375919,33.0,admin.,single,secondary,F,no,no,n,yes
3,338972671,44.0,self-employed,married,secondary,F,no,no,y,yes
4,1472834688,36.0,blue-collar,married,primary,M,yes,no,n,yes
...,...,...,...,...,...,...,...,...,...,...
11157,543198889,40.0,blue-collar,single,unknown,M,no,no,n,yes
11158,458558558,41.0,housemaid,divorced,primary,F,no,no,n,yes
11159,1426334329,59.0,technician,married,primary,F,no,no,y,yes
11160,831035343,42.0,management,married,tertiary,M,no,no,n,no


In [21]:
pd.merge(clients_merged, balances[['client_id','currency']].drop_duplicates(), on = 'client_id')

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,currency
0,249789938,38.0,services,married,secondary,M,yes,no,n,no,CZK
1,1222646323,46.0,services,divorced,unknown,M,no,no,n,no,CZK
2,451375919,33.0,admin.,single,secondary,F,no,no,n,yes,CZK
3,338972671,44.0,self-employed,married,secondary,F,no,no,y,yes,CZK
4,1472834688,36.0,blue-collar,married,primary,M,yes,no,n,yes,CZK
...,...,...,...,...,...,...,...,...,...,...,...
11157,543198889,40.0,blue-collar,single,unknown,M,no,no,n,yes,CZK
11158,458558558,41.0,housemaid,divorced,primary,F,no,no,n,yes,CZK
11159,1426334329,59.0,technician,married,primary,F,no,no,y,yes,CZK
11160,831035343,42.0,management,married,tertiary,M,no,no,n,no,CZK
